# Numerical solving system of linear equations and initial problem for ODE

### Numerical solving system of linear equations

##### Introduction and motivation

One of the basic problems in numerical mathematics is solving system of linear equations. In this essay we will attempt to explore methods for solving quadratic $n\times n$ systems, or system of $n$ equations with $n$ unknowns,

$$
\begin{equation}
a_{11}x_{1} + a_{12}x_{2} + ... + a_{1j}x_{j} + ... + a_{1n}x_{n} = b_{1}\\
a_{21}x_{1} + a_{22}x_{2} + ... + a_{2j}x_{j} + ... + a_{2n}x_{n} = b_{2}\\
\vdots\\
a_{i1}x_{1} + a_{i2}x_{2} + ... + a_{ij}x_{j} + ... + a_{in}x_{n} = b_{i}\\
\vdots\\
a_{n1}x_{1} + a_{n2}x_{2} + ... + a_{nj}x_{j} + ... + a_{nn}x_{n} = b_{n}\\
\end{equation}
$$

We will denote matrix as $A=(a_{ij})_{i,j=1}^{n}$, and refer to it as <b>system matrix</b>, its elements are called <b>coefficients of the system</b>. Vector $b=(b_{i})_{i=1}^{n}$ is called <b>right-hand side</b> vector (RHS). We need to solve for $x$, where $x$ is <b>vector of unknowns</b> $x=(x_{i})_{i=1}^{n}$ such that $Ax=b$ holds.

In theory of linear algebra, solving $Ax=b$ is a almost trivial problem, especially when system matrix is quadratic and regular. Solution $x$ is given by equation $x=A^{-1}b$, where $A^{-1}$ is the inverse matrix for $A$ ($AA^{-1}=A^{-1}A=I$). There are explicit formulae for elements of matrix $A^{-1}$ and for the solution $x$. Besides, the well-known Gaussian elimination solves for $x$ in $O(n^{3})$ elementary operations (where elementary operation is addition, subtraction, multiplication, or division).

We know that the solution $x=A^{-1}b$ exists, it is unique, and there is a simple algorithm that explicitly solves for $x$ using a finite number of simple arithmetic operations.
Sadly, the finite arithmetic of a computer cannot even execute these simple operations exactly, so using Gaussian eliminations (which are simply a finite sequence of formulae which lead to the solution) in general we cannot compute the solution of linear system $Ax=b$ absolutely correct.

For that reason we introduce iterative methods. Iterative methods do not give us a solution, but the approximation of a solution. As it will be shown later, using iterative methods we do not know the exact solution, but we know how close we are to the exact solution.

The methods we will focus on are Jacobi method, Gauss-Seidel method, and SOR method.

#### Base of methods

If we assume $Ax=b$, $det(A)\neq0$, and we keep in mind that instead of exact solution $x$ we are solving for some approximation $\tilde{x}$, we have to have a way to evaluate the quality of the approximation. One way of doing so is, obviously, to calculate the norm of $\delta x$, where $\delta x = \tilde{x} - x$, but that is not possible for $x$ is unknown. The other way is to calculate the residual

$$ r = b - A\tilde{x} \label{eq:1}\tag{1} $$
Let's say that $\tilde{x}$ as a good approximation if, in some norm $||\cdot||$ $\epsilon:=\frac{||r||}{||b||}$ is small enough.

We can justify such criterion by saying that using $\eqref{eq:1}$ we get
$$ A\tilde{x} = \tilde{b}\equiv b-r, \frac{||\tilde{b}-b||}{||b||}=\frac{||r||}{||b||}=\epsilon.$$

##### Jacobi method

$$
A = 
\begin{bmatrix}
    1 & x_1\\
    1 & x_2\\
    1 & x_3\\
    \vdots & \vdots\\
    1 & x_n
\end{bmatrix}, 
z = 
\begin{bmatrix}
a_0\\
a_1\\
\end{bmatrix}, 
b = 
\begin{bmatrix}
y_1\\
y_2\\
y_3\\
\vdots\\
y_n
\end{bmatrix}
$$

Iterative method https://en.wikipedia.org/wiki/Iterative_method
